In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Auntheticate to Hugging Face
!huggingface-cli login

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Generative-ai` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /r

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'Yer lookin\' fer me identity, eh?  Well, matey, I be a swashbucklin\' chatbot, sailin\' the seven seas o\' knowledge. Me name be "Blackbeak," and I be here to provide ye with the treasure o\' information ye seek. I\'ve got a wealth o\' wisdom at me disposal, and I be willin\' to share it with ye, savvy? So hoist the colors and set course fer a conversation with Blackbeak!'}


In [1]:
# 🚀 Install dependencies
!pip install --upgrade transformers accelerate torch gradio

import torch
import transformers
import gradio as gr
from threading import Thread
from huggingface_hub import login

# =========================
# 🔑 Hugging Face Authentication
# =========================
HF_TOKEN = ""  # <-- paste your HF token
login(HF_TOKEN)

# =========================
# ⚡ Load Model
# =========================
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# =========================
# 🧠 Chat Function with Streaming
# =========================
def generate_reply(user_message, history, mode):
    # Base system role
    system_prompt = "You are a helpful AI assistant."
    if mode == "Pirate Mode":
        system_prompt = "You are a pirate chatbot who always responds in pirate speak!"

    # Build conversation
    messages = [{"role": "system", "content": system_prompt}]
    for human, ai in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": ai})
    messages.append({"role": "user", "content": user_message})

    # Convert to model input
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    # Streaming setup
    streamer = transformers.TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = dict(
        inputs=inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        streamer=streamer,
    )

    # Run generation in background thread
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Collect tokens as they stream
    partial_text = ""
    for new_text in streamer:
        partial_text += new_text
        yield partial_text  # 🚀 stream to UI

# =========================
# 🎨 Gradio UI
# =========================
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="pink")) as demo:
    gr.Markdown(
        """
        # 🧑‍💻 Friendly Chatbot with Streaming
        Toggle between **Pirate Mode 🏴‍☠️** and **Normal Mode 🤖**
        """
    )

    with gr.Row():
        mode = gr.Radio(["Normal Mode", "Pirate Mode"], value="Normal Mode", label="Choose Chat Mode")

    chatbot_ui = gr.Chatbot(height=400, bubble_full_width=False)
    msg = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    clear = gr.Button("🧹 Clear Chat")

    def respond(message, chat_history, mode):
        bot_stream = generate_reply(message, chat_history, mode)
        chat_history.append((message, ""))  # placeholder
        for partial_text in bot_stream:
            chat_history[-1] = (message, partial_text)
            yield "", chat_history

    msg.submit(respond, [msg, chatbot_ui, mode], [msg, chatbot_ui])
    clear.click(lambda: None, None, chatbot_ui, queue=False)

# =========================
# 🚀 Launch
# =========================
demo.launch(share=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.13.0
    Uninstalling gradio_client-1.13.0:
      Successfully uninstalled gradio_client-1.13.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.46.0
    Uninstalling gradio-5.46.0:
      Successfully uninstalled gradio-5.46.0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

/tmp/ipython-input-4082583808.py:83: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(height=400, bubble_full_width=False)
/tmp/ipython-input-4082583808.py:83: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_ui = gr.Chatbot(height=400, bubble_full_width=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8afd27f5efea4aa1ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
